# Notebook 75: 学習ループの完成 ― SGDで重みを更新

## Training Loop: Parameter Updates with SGD

---

### このノートブックの位置づけ

**Unit 0.0「ニューラルエンジンの深部」** の第6章として、計算グラフ・逆伝播を統合し、**完全な学習ループ** を実装します。

### 学習目標

1. **SGD（確率的勾配降下法）** とそのバリエーションを実装する
2. **学習ループ** を構成する要素を理解する
3. **XOR問題** を解いて動作を確認する
4. **学習曲線** と **決定境界** を可視化する

### 前提知識

- Notebook 70-74 の内容

---

## 目次

1. [学習アルゴリズムの概要](#1-学習アルゴリズムの概要)
2. [オプティマイザの実装](#2-オプティマイザの実装)
3. [学習ループの実装](#3-学習ループの実装)
4. [XOR問題を解く](#4-xor問題を解く)
5. [学習の可視化](#5-学習の可視化)
6. [より複雑な問題への適用](#6-より複雑な問題への適用)
7. [演習問題](#7-演習問題)
8. [まとめと次のステップ](#8-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 0. 前のノートブックのクラスを再定義

In [ ]:
# Notebook 74 の Layer クラス群を再定義

class Layer(ABC):
    def __init__(self):
        self.params = {}
        self.grads = {}
        self.cache = {}
    
    @abstractmethod
    def forward(self, x): pass
    
    @abstractmethod
    def backward(self, dout): pass


class Linear(Layer):
    def __init__(self, in_features, out_features):
        super().__init__()
        scale = np.sqrt(2.0 / (in_features + out_features))
        self.params['W'] = np.random.randn(in_features, out_features) * scale
        self.params['b'] = np.zeros(out_features)
        self.grads['W'] = None
        self.grads['b'] = None
    
    def forward(self, x):
        self.cache['x'] = x
        return x @ self.params['W'] + self.params['b']
    
    def backward(self, dout):
        x = self.cache['x']
        self.grads['W'] = x.T @ dout
        self.grads['b'] = np.sum(dout, axis=0)
        return dout @ self.params['W'].T


class Sigmoid(Layer):
    def forward(self, x):
        y = 1 / (1 + np.exp(-np.clip(x, -500, 500)))
        self.cache['y'] = y
        return y
    
    def backward(self, dout):
        y = self.cache['y']
        return dout * y * (1 - y)


class ReLU(Layer):
    def forward(self, x):
        self.cache['mask'] = (x > 0).astype(float)
        return np.maximum(0, x)
    
    def backward(self, dout):
        return dout * self.cache['mask']


class Tanh(Layer):
    def forward(self, x):
        y = np.tanh(x)
        self.cache['y'] = y
        return y
    
    def backward(self, dout):
        y = self.cache['y']
        return dout * (1 - y ** 2)


class MSELoss:
    def __init__(self):
        self.cache = {}
    
    def forward(self, y, t):
        self.cache['y'] = y
        self.cache['t'] = t
        self.cache['N'] = y.shape[0]
        return np.mean((y - t) ** 2)
    
    def backward(self):
        y, t, N = self.cache['y'], self.cache['t'], self.cache['N']
        return (2 / N) * (y - t)


class MLP:
    def __init__(self, layer_dims, activation='relu'):
        self.layers = []
        act_class = {'relu': ReLU, 'sigmoid': Sigmoid, 'tanh': Tanh}[activation]
        
        for i in range(len(layer_dims) - 1):
            self.layers.append(Linear(layer_dims[i], layer_dims[i+1]))
            if i < len(layer_dims) - 2:
                self.layers.append(act_class())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, dout):
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def get_params(self):
        params = []
        for layer in self.layers:
            if hasattr(layer, 'params'):
                for name, param in layer.params.items():
                    params.append((layer, name, param))
        return params


print("Layer クラス群を定義しました")

---

## 1. 学習アルゴリズムの概要

### 1.1 勾配降下法の基本

$$
\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)
$$

- $\theta$: パラメータ（重み、バイアス）
- $\eta$: 学習率（Learning Rate）
- $\nabla L$: 損失の勾配

### 1.2 学習ループの構成要素

```
for epoch in range(num_epochs):
    for batch in data:
        1. 順伝播: y = model(x)
        2. 損失計算: L = loss(y, t)
        3. 逆伝播: grads = backward(L)
        4. パラメータ更新: params -= lr * grads
```

---

## 2. オプティマイザの実装

### 2.1 SGD（確率的勾配降下法）

In [ ]:
class SGD:
    """
    確率的勾配降下法（Stochastic Gradient Descent）
    
    更新則: θ = θ - η * ∇L
    """
    
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, model):
        """モデルのパラメータを更新"""
        for layer in model.layers:
            if hasattr(layer, 'params'):
                for name in layer.params:
                    if layer.grads[name] is not None:
                        layer.params[name] -= self.lr * layer.grads[name]


print("SGDオプティマイザを定義しました")

### 2.2 Momentum SGD

In [ ]:
class MomentumSGD:
    """
    モーメンタム付きSGD
    
    更新則:
        v = μ * v - η * ∇L
        θ = θ + v
    
    μ: モーメンタム係数（通常0.9）
    """
    
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.velocity = {}  # 各パラメータの速度を保持
    
    def update(self, model):
        for layer in model.layers:
            if hasattr(layer, 'params'):
                for name in layer.params:
                    if layer.grads[name] is not None:
                        key = id(layer.params[name])
                        
                        # 速度の初期化
                        if key not in self.velocity:
                            self.velocity[key] = np.zeros_like(layer.params[name])
                        
                        # 速度の更新
                        self.velocity[key] = self.momentum * self.velocity[key] - self.lr * layer.grads[name]
                        
                        # パラメータの更新
                        layer.params[name] += self.velocity[key]


print("MomentumSGDオプティマイザを定義しました")

### 2.3 Adam

In [ ]:
class Adam:
    """
    Adam (Adaptive Moment Estimation)
    
    更新則:
        m = β1 * m + (1 - β1) * g
        v = β2 * v + (1 - β2) * g²
        m_hat = m / (1 - β1^t)
        v_hat = v / (1 - β2^t)
        θ = θ - η * m_hat / (√v_hat + ε)
    """
    
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = {}  # 1次モーメント
        self.v = {}  # 2次モーメント
        self.t = 0   # タイムステップ
    
    def update(self, model):
        self.t += 1
        
        for layer in model.layers:
            if hasattr(layer, 'params'):
                for name in layer.params:
                    if layer.grads[name] is not None:
                        key = id(layer.params[name])
                        g = layer.grads[name]
                        
                        # 初期化
                        if key not in self.m:
                            self.m[key] = np.zeros_like(g)
                            self.v[key] = np.zeros_like(g)
                        
                        # モーメントの更新
                        self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * g
                        self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * (g ** 2)
                        
                        # バイアス補正
                        m_hat = self.m[key] / (1 - self.beta1 ** self.t)
                        v_hat = self.v[key] / (1 - self.beta2 ** self.t)
                        
                        # パラメータ更新
                        layer.params[name] -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)


print("Adamオプティマイザを定義しました")

---

## 3. 学習ループの実装

In [ ]:
def train(model, loss_fn, optimizer, X, t, epochs=1000, verbose=True, log_interval=100):
    """
    学習ループ
    
    Args:
        model: MLPモデル
        loss_fn: 損失関数
        optimizer: オプティマイザ
        X: 入力データ (N, D)
        t: ターゲット (N, K)
        epochs: エポック数
        verbose: 進捗表示
        log_interval: ログ出力間隔
    
    Returns:
        学習履歴（損失のリスト）
    """
    history = {'loss': []}
    
    for epoch in range(epochs):
        # 1. 順伝播
        y = model.forward(X)
        
        # 2. 損失計算
        loss = loss_fn.forward(y, t)
        history['loss'].append(loss)
        
        # 3. 逆伝播
        dout = loss_fn.backward()
        model.backward(dout)
        
        # 4. パラメータ更新
        optimizer.update(model)
        
        # ログ出力
        if verbose and (epoch + 1) % log_interval == 0:
            print(f"Epoch {epoch+1:5d}: Loss = {loss:.6f}")
    
    return history


print("学習ループを定義しました")

---

## 4. XOR問題を解く

### 4.1 XOR問題とは

XOR（排他的論理和）は線形分離不可能な問題で、ニューラルネットワークの能力を示す古典的なベンチマークです。

```
入力        出力
(0, 0) →    0
(0, 1) →    1
(1, 0) →    1
(1, 1) →    0
```

In [ ]:
# XORデータセット
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float64)
t_xor = np.array([[0], [1], [1], [0]], dtype=np.float64)

print("【XORデータセット】")
print("入力 → 出力")
for x, y in zip(X_xor, t_xor):
    print(f"  {x} → {y[0]:.0f}")

# 可視化
fig, ax = plt.subplots(figsize=(6, 6))
colors = ['red' if y[0] == 0 else 'blue' for y in t_xor]
ax.scatter(X_xor[:, 0], X_xor[:, 1], c=colors, s=200, edgecolors='black', linewidth=2)
for x, y in zip(X_xor, t_xor):
    ax.annotate(f'{int(y[0])}', (x[0], x[1]), fontsize=14, ha='center', va='center', color='white', fontweight='bold')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_title('XOR問題: 線形分離不可能')
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-0.5, 1.5)
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# XOR問題を解く
np.random.seed(42)

# モデル: 2 → 4 → 1
model = MLP([2, 4, 1], activation='sigmoid')
loss_fn = MSELoss()
optimizer = SGD(lr=1.0)

print("【XOR問題の学習】")
print(f"モデル: 2 → 4 → 1 (Sigmoid)")
print(f"オプティマイザ: SGD (lr=1.0)")
print()

# 学習
history = train(model, loss_fn, optimizer, X_xor, t_xor, epochs=5000, log_interval=1000)

# 最終結果
print("\n【学習後の予測】")
y_pred = model.forward(X_xor)
for x, t, y in zip(X_xor, t_xor, y_pred):
    pred_class = 1 if y[0] > 0.5 else 0
    correct = "✓" if pred_class == t[0] else "✗"
    print(f"  {x} → 予測: {y[0]:.4f} ({pred_class}) 正解: {int(t[0])} {correct}")

---

## 5. 学習の可視化

### 5.1 学習曲線

In [ ]:
# 学習曲線の可視化
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history['loss'], linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('学習曲線: XOR問題')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)
plt.show()

### 5.2 決定境界の可視化

In [ ]:
def plot_decision_boundary(model, X, t, title="Decision Boundary"):
    """決定境界を可視化"""
    h = 0.01
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.forward(grid)
    Z = Z.reshape(xx.shape)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # 決定境界（確率のコンター）
    contour = ax.contourf(xx, yy, Z, levels=20, cmap='RdBu', alpha=0.8)
    plt.colorbar(contour, ax=ax, label='出力値')
    
    # 決定境界線（0.5）
    ax.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
    
    # データ点
    colors = ['red' if y[0] == 0 else 'blue' for y in t]
    ax.scatter(X[:, 0], X[:, 1], c=colors, s=200, edgecolors='black', linewidth=2, zorder=5)
    
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title(title)
    
    plt.tight_layout()
    return fig, ax


plot_decision_boundary(model, X_xor, t_xor, "XOR問題の決定境界")
plt.show()

### 5.3 オプティマイザの比較

In [ ]:
# 異なるオプティマイザの比較
np.random.seed(42)

optimizers = {
    'SGD (lr=1.0)': SGD(lr=1.0),
    'Momentum (lr=0.5)': MomentumSGD(lr=0.5, momentum=0.9),
    'Adam (lr=0.1)': Adam(lr=0.1),
}

histories = {}

for name, optimizer in optimizers.items():
    np.random.seed(42)  # 同じ初期値
    model = MLP([2, 4, 1], activation='sigmoid')
    loss_fn = MSELoss()
    
    history = train(model, loss_fn, optimizer, X_xor, t_xor, epochs=2000, verbose=False)
    histories[name] = history['loss']

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))
for name, losses in histories.items():
    ax.plot(losses, label=name, linewidth=2)

ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('オプティマイザの比較: XOR問題')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

---

## 6. より複雑な問題への適用

### 6.1 円形データセット

In [ ]:
# 円形に分布するデータを生成
def make_circles(n_samples=200, noise=0.1):
    np.random.seed(42)
    
    # 内側の円
    n_inner = n_samples // 2
    theta_inner = np.random.uniform(0, 2 * np.pi, n_inner)
    r_inner = np.random.normal(0.3, noise, n_inner)
    X_inner = np.column_stack([r_inner * np.cos(theta_inner), r_inner * np.sin(theta_inner)])
    
    # 外側の円
    n_outer = n_samples - n_inner
    theta_outer = np.random.uniform(0, 2 * np.pi, n_outer)
    r_outer = np.random.normal(1.0, noise, n_outer)
    X_outer = np.column_stack([r_outer * np.cos(theta_outer), r_outer * np.sin(theta_outer)])
    
    X = np.vstack([X_inner, X_outer])
    t = np.vstack([np.zeros((n_inner, 1)), np.ones((n_outer, 1))])
    
    return X, t


X_circle, t_circle = make_circles(200, noise=0.1)

# 可視化
fig, ax = plt.subplots(figsize=(6, 6))
colors = ['red' if y[0] == 0 else 'blue' for y in t_circle]
ax.scatter(X_circle[:, 0], X_circle[:, 1], c=colors, alpha=0.7)
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_title('円形データセット')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 円形データセットの学習
np.random.seed(42)

model = MLP([2, 16, 8, 1], activation='relu')
loss_fn = MSELoss()
optimizer = Adam(lr=0.01)

print("【円形データセットの学習】")
history = train(model, loss_fn, optimizer, X_circle, t_circle, epochs=1000, log_interval=200)

# 決定境界の可視化
plot_decision_boundary(model, X_circle, t_circle, "円形データセットの決定境界")
plt.show()

---

## 7. 演習問題

### 演習 7.1: 学習率の影響

学習率を変えて学習曲線の違いを観察してください。

In [ ]:
# 演習 7.1: 解答欄

# TODO: 学習率 0.01, 0.1, 1.0, 5.0 で学習を比較

pass

### 演習 7.2: ネットワーク構造の影響

隠れ層のニューロン数を変えて、学習結果を比較してください。

In [ ]:
# 演習 7.2: 解答欄

# TODO: [2, 2, 1], [2, 4, 1], [2, 8, 1], [2, 4, 4, 1] などを比較

pass

---

## 8. まとめと次のステップ

### このノートブックで学んだこと

1. **オプティマイザ**: SGD, Momentum, Adam の実装

2. **学習ループ**: 順伝播 → 損失計算 → 逆伝播 → パラメータ更新

3. **XOR問題**: 線形分離不可能な問題を多層パーセプトロンで解く

4. **可視化**: 学習曲線、決定境界

### 次のノートブック（76: 勾配の病理学）への橋渡し

学習がうまくいかない場合があります。次のノートブックでは：

- **勾配消失・勾配爆発** の問題
- **鞍点と局所解** の罠
- **診断と対策** の方法